# **Big Data Project - Spotify**

The purpose of the project is to implement a big data system that collects, analyses (and possibly predicts) the song’s popularity by looking at Spotify data (accessed with WebApi) in various countries, exploit implications of Covid-19 data to understand if making us staying at home influence music preferences.

This notebook contains the **Analysis** part of the project.
The output of this system will be on one side the processed audio features for each country and on the other side a set of visualization following the Pleasure-Arousal approach. It is model proposed by [Russell in 1980](https://www.researchgate.net/publication/235361517_A_Circumplex_Model_of_Affect) to express human emotions in a two-dimension space, arousal and pleasure. 

In addition. it has been shown that these two dimensions can also be interpreted in audio features. Specifically, energy and valence can act this two dimensions space. As literally stated by [Helmholz in 2017](https://www.researchgate.net/publication/320555005_Summer_hot_Winter_not_-_Seasonal_influences_on_context-based_music_recommendations) "Energy is measured from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. On the other hand, tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)"

---


The fist dataset to load corresponds to the weekly charts, this is the top 200 songs for each country from 2017 to 2020. This data was gathered from the site [spotifycharts.com](https://spotifycharts.com/regional), retrieved from the DynamoDB instance and stored in the spark table `weekly_charts_table`

In [3]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext

charts_df = spark.table("weekly_charts_table")
charts_df.show(3)
charts_df.printSchema()

+-------+----------+--------------------+
country| day| songs|
+-------+----------+--------------------+
 global|2018-11-01|[[st -> 40931264,...|
 global|2018-11-08|[[st -> 47098337,...|
 global|2018-11-15|[[st -> 59975503,...|
+-------+----------+--------------------+
only showing top 3 rows

root
-- country: string (nullable = true)
-- day: string (nullable = true)
-- songs: array (nullable = true)
 |-- element: map (containsNull = true)
 | |-- key: string
 | |-- value: string (valueContainsNull = true)

The second dataset contains the audio feaures data for each song, this is data collected from the Spotify WebApi, specifically from the [audio features endpoint ](https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/)

In [5]:
audio_df = spark.table("song_features_table")
audio_df.show()
audio_df.printSchema()

+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
acousticness|danceability|duration_ms|energy|instrumentalness|key|liveness|loudness|mode| s_id|speechiness| tempo|time_signature|valence|
+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
 0.841| 0.304| 173321| 0.28| 0.0| 2| 0.0825| -9.66| 0|77kmchibFTQvcwQJC...| 0.0334|116.725| 4| 0.108|
 0.257| 0.751| 162528| 0.756| 0.0| 9| 0.0582| -5.453| 0|5tRa5hnltiyqY63rW...| 0.0375|129.954| 4| 0.644|
 0.127| 0.675| 299960| 0.771| 6.16E-6| 8| 0.0935| -5.062| 1|0sOpGWXWkxNDev51D...| 0.0282|132.038| 4| 0.619|
 0.0351| 0.477| 204984| 0.755| 0.0| 10| 0.1| -5.781| 0|1ejlLAA420azVRdgY...| 0.163| 81.283| 4| 0.586|
 0.0225| 0.665| 176121| 0.631| 0.0| 0| 0.305| -6.365| 1|7ad3XBdqiGoGIuJ83...| 0.0402|141.093| 4| 0.581|
 0.412| 0.765| 185654| 0.751| 0.0| 1| 0.169| -4.884| 0|2NpuceKgvkdmHG3vJ...| 0.236|148.028| 4| 0.61|
 0.589| 0.868| 236388| 0.751| 0.0| 7| 0.112| -5.426| 1|7LQtFOR1rnk9T8cKt...| 0.0738|127.973| 4| 0.785|
 0.072| 0.724| 177600| 0.646| 1.57E-4| 10| 0.0923| -6.475| 1|3GSMdtJphymHEsR8K...| 0.0352|112.019| 4| 0.957|
 0.0539| 0.671| 159209| 0.748| 1.8E-6| 9| 0.102| -6.47| 0|4rqMsKk1nEjL275UH...| 0.29| 96.29| 4| 0.192|
 0.112| 0.585| 247466| 0.735| 0.0| 10| 0.301| -5.206| 1|3fhvRxPQ1r7DP0JVQ...| 0.0479|111.058| 4| 0.405|
 0.539| 0.785| 189414| 0.701| 1.09E-4| 10| 0.854| -5.702| 1|4RdYKxFtC8j35NEi2...| 0.038|103.004| 4| 0.498|
 0.294| 0.655| 158907| 0.806| 0.0| 4| 0.113| -5.317| 0|56QinE0AmuVHDQGqb...| 0.219| 96.797| 4| 0.752|
 0.224| 0.678| 186680| 0.739| 0.0| 1| 0.0625| -4.479| 1|0qqUNsGRZVT9dlru6...| 0.0733|118.016| 4| 0.575|
 0.42| 0.663| 210000| 0.69| 0.0| 11| 0.11| -4.871| 0|0d4PvVcC2GxJoVm76...| 0.266|129.832| 4| 0.715|
 0.68| 0.831| 220328| 0.741| 1.04E-6| 0| 0.104| -5.866| 1|7nY4dQPdqg5apceI2...| 0.043|121.995| 4| 0.787|
 0.398| 0.747| 169090| 0.505| 3.05E-5| 4| 0.0868| -11.173| 0|4HjRpvW35frIlT33q...| 0.366|148.107| 4| 0.757|
 0.093| 0.847| 165504| 0.608| 0.0| 11| 0.132| -7.298| 1|0GrklkkWttHKbQ8XX...| 0.195|100.005| 4| 0.668|
 0.0392| 0.675| 201240| 0.662| 0.0| 1| 0.128| -4.989| 1|73NbHgUZ51H65Bfo2...| 0.0318|134.936| 4| 0.536|
 0.00242| 0.746| 207660| 0.699| 0.0| 1| 0.363| -9.243| 1|76gf6DpUXDvgXnGsI...| 0.0705| 141.02| 4| 0.466|
 0.226| 0.692| 247164| 0.86| 0.0| 7| 0.119| -4.257| 0|5xx3UTSo9aevmdGI8...| 0.437|133.885| 4| 0.566|
+------------+------------+-----------+------+----------------+---+--------+--------+----+--------------------+-----------+-------+--------------+-------+
only showing top 20 rows

root
-- acousticness: double (nullable = true)
-- danceability: double (nullable = true)
-- duration_ms: long (nullable = true)
-- energy: double (nullable = true)
-- instrumentalness: double (nullable = true)
-- key: long (nullable = true)
-- liveness: double (nullable = true)
-- loudness: double (nullable = true)
-- mode: long (nullable = true)
-- s_id: string (nullable = true)
-- speechiness: double (nullable = true)
-- tempo: double (nullable = true)
-- time_signature: long (nullable = true)
-- valence: double (nullable = true)

Filter the audio features that are not relevant to the analysis such as `duration_ms` and those that correspond to categorical data such as `mode` and `key`

In [7]:
audio_df = audio_df.select("s_id","acousticness","danceability","energy","liveness","loudness","speechiness","tempo","valence")

### **Covid-19 Data Processing**

One of the goals of the project is exploring the implications of covid-19 in the listening preferences by country. The selected dataset corresponds to **total confirmed cases** found in [Our World in Data](https://ourworldindata.org/coronavirus-source-data) 

This dataset is read with pandas because the data is not large, and it will be useful for the final plots

In [9]:
import pandas as pd

cases_df = pd.read_csv("/dbfs/FileStore/tables/total_cases.csv")
cases_df.tail()

,date,World,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bonaire Sint Eustatius and Saba,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,...,Sint Maarten (Dutch part),Slovakia,Slovenia,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Swaziland,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Turks and Caicos Islands,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,United States Virgin Islands,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Western Sahara,Yemen,Zambia,Zimbabwe
175,2020-06-23,9063774,29143.0,1995.0,11920.0,855.0,183.0,3.0,26.0,44918.0,20588.0,101.0,7474,17320,13207.0,104.0,22407.0,115786.0,97.0,59023.0,60910,23.0,807.0,146.0,69.0,25493.0,7.0,3524.0,89.0,1106470,8.0,141.0,3984.0,907.0,144.0,130.0,11892.0,101637,944.0,195.0,...,77.0,1588.0,1521.0,2812.0,101590.0,12484,1892.0,246752.0,1951.0,8698.0,319.0,643.0,58932,31227,219.0,446.0,5513.0,509.0,3151.0,24.0,569.0,123.0,1159.0,188897.0,14.0,774.0,37241.0,45303.0,305289,2312302,76.0,882.0,6500.0,12.0,4048.0,349.0,25.0,967.0,1430.0,512.0
176,2020-06-24,9229573,29481.0,2047.0,12076.0,855.0,186.0,3.0,26.0,44918.0,21717.0,101.0,7492,17351,13715.0,104.0,23062.0,119198.0,97.0,59487.0,61000,23.0,850.0,146.0,70.0,26389.0,7.0,3587.0,89.0,1145906,8.0,141.0,4114.0,919.0,144.0,130.0,12041.0,101963,982.0,195.0,...,77.0,1589.0,1534.0,2835.0,106108.0,12535,1916.0,247086.0,1991.0,8889.0,319.0,674.0,60837,31249,231.0,446.0,5567.0,509.0,3156.0,24.0,576.0,123.0,1159.0,190165.0,14.0,797.0,39014.0,45683.0,306210,2347022,76.0,885.0,6755.0,12.0,4186.0,349.0,25.0,992.0,1477.0,525.0
177,2020-06-25,9400810,29715.0,2114.0,12248.0,855.0,189.0,3.0,65.0,49838.0,21717.0,101.0,7521,17384,14305.0,104.0,23570.0,122660.0,97.0,59945.0,61013,23.0,902.0,146.0,70.0,27487.0,7.0,3675.0,89.0,1188631,8.0,141.0,4242.0,934.0,144.0,130.0,12270.0,102242,999.0,196.0,...,77.0,1607.0,1541.0,2860.0,111796.0,12563,1930.0,247486.0,2001.0,8889.0,357.0,690.0,62324,31293,231.0,446.0,5630.0,509.0,3158.0,24.0,583.0,123.0,1160.0,191657.0,15.0,805.0,39014.0,46133.0,306862,2381361,80.0,902.0,6990.0,12.0,4365.0,352.0,25.0,1015.0,1489.0,530.0
178,2020-06-26,9582321,30175.0,2192.0,12445.0,855.0,197.0,3.0,65.0,52444.0,23247.0,101.0,7558,17431,14852.0,104.0,24081.0,126606.0,97.0,60382.0,61106,23.0,1017.0,146.0,70.0,28503.0,7.0,3796.0,89.0,1228114,8.0,141.0,4408.0,941.0,144.0,130.0,12592.0,102611,1003.0,196.0,...,77.0,1630.0,1547.0,2878.0,118375.0,12602,1942.0,247905.0,2010.0,8984.0,373.0,706.0,63890,31345,242.0,447.0,5691.0,509.0,3158.0,24.0,588.0,123.0,1162.0,193115.0,15.0,821.0,40008.0,46563.0,307980,2422310,81.0,907.0,7228.0,12.0,4563.0,352.0,25.0,1076.0,1497.0,551.0
179,2020-06-27,9770954,30451.0,2269.0,12685.0,855.0,212.0,3.0,65.0,52444.0,23909.0,101.0,7595,17498,15369.0,104.0,24805.0,130474.0,97.0,60713.0,61106,24.0,1053.0,146.0,75.0,29423.0,7.0,3934.0,125.0,1274974,8.0,141.0,4513.0,941.0,144.0,139.0,12592.0,102783,1027.0,196.0,...,77.0,1643.0,1558.0,2878.0,124590.0,12653,1952.0,NaN,2014.0,9257.0,391.0,728.0,65137,31403,255.0,447.0,5747.0,509.0,3162.0,24.0,591.0,124.0,1164.0,194511.0,15.0,833.0,41117.0,46973.0,309360,2467837,81.0,919.0,7490.0,12.0,4779.0,353.0,25.0,1089.0,1531.0,561.0


The total confirmed cases dataframe contains more countries than the ones available in the charts So it is needed to drop the columns of the countries for which we dont have charts. First, we get the list of unique country codes in the charts dataframe

In [11]:
chart_countries = [str(row.country) for row in charts_df.select("country").distinct().collect()]
chart_countries[:10]

Out[8]: ['cr', 'us', 'il', 'cl', 'ro', 'jp', 'lv', 'vn', 'pl', 'za']

`country_columns` corresponds to the list of countries in the covid-19 dataframe

In [13]:
country_columns = list(cases_df.columns)
country_columns[:20]

Out[10]: ['date',
 'World',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus']

On one side, we have country codes in the charts dataframe, but on the other hand, the covid-19 dataframe contains the actual names of the countries. So it is necessary to transform the names to codes and filter. Fortunately, the library [country-converter](https://pypi.org/project/country-converter/) makes these transformations.

In [15]:
import country_converter as coco
names_dict = {country_name: coco.convert(names=country_name, to='iso2').lower() for country_name in country_columns}
names_dict["World"] = "global"
names_dict

WARNING:root:date not found in regex
WARNING:root:World not found in regex
WARNING:root:International not found in regex
WARNING:root:Timor not found in regex
Out[11]: {'date': 'not found',
 'World': 'global',
 'Afghanistan': 'af',
 'Albania': 'al',
 'Algeria': 'dz',
 'Andorra': 'ad',
 'Angola': 'ao',
 'Anguilla': 'ai',
 'Antigua and Barbuda': 'ag',
 'Argentina': 'ar',
 'Armenia': 'am',
 'Aruba': 'aw',
 'Australia': 'au',
 'Austria': 'at',
 'Azerbaijan': 'az',
 'Bahamas': 'bs',
 'Bahrain': 'bh',
 'Bangladesh': 'bd',
 'Barbados': 'bb',
 'Belarus': 'by',
 'Belgium': 'be',
 'Belize': 'bz',
 'Benin': 'bj',
 'Bermuda': 'bm',
 'Bhutan': 'bt',
 'Bolivia': 'bo',
 'Bonaire Sint Eustatius and Saba': 'bq',
 'Bosnia and Herzegovina': 'ba',
 'Botswana': 'bw',
 'Brazil': 'br',
 'British Virgin Islands': 'vg',
 'Brunei': 'bn',
 'Bulgaria': 'bg',
 'Burkina Faso': 'bf',
 'Burundi': 'bi',
 'Cambodia': 'kh',
 'Cameroon': 'cm',
 'Canada': 'ca',
 'Cape Verde': 'cv',
 'Cayman Islands': 'ky',
 'Central African Republic': 'cf',
 'Chad': 'td',
 'Chile': 'cl',
 'China': 'cn',
 'Colombia': 'co',
 'Comoros': 'km',
 'Congo': 'cg',
 'Costa Rica': 'cr',
 "Cote d'Ivoire": 'ci',
 'Croatia': 'hr',
 'Cuba': 'cu',
 'Curacao': 'cw',
 'Cyprus': 'cy',
 'Czech Republic': 'cz',
 'Democratic Republic of Congo': 'cd',
 'Denmark': 'dk',
 'Djibouti': 'dj',
 'Dominica': 'dm',
 'Dominican Republic': 'do',
 'Ecuador': 'ec',
 'Egypt': 'eg',
 'El Salvador': 'sv',
 'Equatorial Guinea': 'gq',
 'Eritrea': 'er',
 'Estonia': 'ee',
 'Ethiopia': 'et',
 'Faeroe Islands': 'fo',
 'Falkland Islands': 'fk',
 'Fiji': 'fj',
 'Finland': 'fi',
 'France': 'fr',
 'French Polynesia': 'pf',
 'Gabon': 'ga',
 'Gambia': 'gm',
 'Georgia': 'ge',
 'Germany': 'de',
 'Ghana': 'gh',
 'Gibraltar': 'gi',
 'Greece': 'gr',
 'Greenland': 'gl',
 'Grenada': 'gd',
 'Guam': 'gu',
 'Guatemala': 'gt',
 'Guernsey': 'gg',
 'Guinea': 'gn',
 'Guinea-Bissau': 'gw',
 'Guyana': 'gy',
 'Haiti': 'ht',
 'Honduras': 'hn',
 'Hungary': 'hu',
 'Iceland': 'is',
 'India': 'in',
 'Indonesia': 'id',
 'International': 'not found',
 'Iran': 'ir',
 'Iraq': 'iq',
 'Ireland': 'ie',
 'Isle of Man': 'im',
 'Israel': 'il',
 'Italy': 'it',
 'Jamaica': 'jm',
 'Japan': 'jp',
 'Jersey': 'je',
 'Jordan': 'jo',
 'Kazakhstan': 'kz',
 'Kenya': 'ke',
 'Kosovo': 'xk',
 'Kuwait': 'kw',
 'Kyrgyzstan': 'kg',
 'Laos': 'la',
 'Latvia': 'lv',
 'Lebanon': 'lb',
 'Lesotho': 'ls',
 'Liberia': 'lr',
 'Libya': 'ly',
 'Liechtenstein': 'li',
 'Lithuania': 'lt',
 'Luxembourg': 'lu',
 'Macedonia': 'mk',
 'Madagascar': 'mg',
 'Malawi': 'mw',
 'Malaysia': 'my',
 'Maldives': 'mv',
 'Mali': 'ml',
 'Malta': 'mt',
 'Mauritania': 'mr',
 'Mauritius': 'mu',
 'Mexico': 'mx',
 'Moldova': 'md',
 'Monaco': 'mc',
 'Mongolia': 'mn',
 'Montenegro': 'me',
 'Montserrat': 'ms',
 'Morocco': 'ma',
 'Mozambique': 'mz',
 'Myanmar': 'mm',
 'Namibia': 'na',
 'Nepal': 'np',
 'Netherlands': 'nl',
 'New Caledonia': 'nc',
 'New Zealand': 'nz',
 'Nicaragua': 'ni',
 'Niger': 'ne',
 'Nigeria': 'ng',
 'Northern Mariana Islands': 'mp',
 'Norway': 'no',
 'Oman': 'om',
 'Pakistan': 'pk',
 'Palestine': 'ps',
 'Panama': 'pa',
 'Papua New Guinea': 'pg',
 'Paraguay': 'py',
 'Peru': 'pe',
 'Philippines': 'ph',
 'Poland': 'pl',
 'Portugal': 'pt',
 'Puerto Rico': 'pr',
 'Qatar': 'qa',
 'Romania': 'ro',
 'Russia': 'ru',
 'Rwanda': 'rw',
 'Saint Kitts and Nevis': 'kn',
 'Saint Lucia': 'lc',
 'Saint Vincent and the Grenadines': 'vc',
 'San Marino': 'sm',
 'Sao Tome and Principe': 'st',
 'Saudi Arabia': 'sa',
 'Senegal': 'sn',
 'Serbia': 'rs',
 'Seychelles': 'sc',
 'Sierra Leone': 'sl',
 'Singapore': 'sg',
 'Sint Maarten (Dutch part)': 'sx',
 'Slovakia': 'sk',
 'Slovenia': 'si',
 'Somalia': 'so',
 'South Africa': 'za',
 'South Korea': 'kr',
 'South Sudan': 'ss',
 'Spain': 'es',
 'Sri Lanka': 'lk',
 'Sudan': 'sd',
 'Suriname': 'sr',
 'Swaziland': 'sz',
 'Sweden': 'se',
 'Switzerland': 'ch',
 'Syria': 'sy',
 'Taiwan': 'tw',
 'Tajikistan': 'tj',
 'Tanzania': 'tz',
 'Thailand': 'th',
 'Timor': 'not found',
 'Togo':

In [16]:
names_dict["date"] = "date"
cases_df = cases_df.rename(columns=names_dict)

Now that we have country codes in both sides, we can filter the covid-19 cases dataframe to keep only those countries that have charts information.

In [18]:
chart_countries.append("date")
drop_columns = [c for c in list(cases_df.columns) if c not in chart_countries]
cases_df = cases_df.drop(columns=drop_columns)
cases_df.tail()

,date,global,ar,au,at,be,bo,br,bg,ca,cl,co,cr,cz,dk,do,ec,sv,ee,fi,fr,de,gr,gt,hn,hu,is,in,id,ie,il,it,jp,lv,lt,lu,my,mt,mx,nl,nz,ni,no,pa,py,pe,ph,pl,pt,ro,sg,sk,za,es,se,ch,tw,th,tr,gb,us,uy,vn
175,2020-06-23,9063774,44918.0,7474,17320,60910,25493.0,1106470,3984.0,101637,246963.0,71183.0,2277.0,10561,12527,27370.0,50640.0,4808.0,1981.0,7144.0,160750,190862,3287.0,13769.0,13356.0,4107.0,1823,440215.0,46845.0,25383.0,21082.0,238720,17968,1111.0,1801.0,4121.0,8587.0,665.0,185122.0,49658,1165.0,2014.0,8745,26752.0,1392.0,254936.0,30682.0,32227.0,39392.0,24291.0,42313,1588.0,101590.0,246752.0,58932,31227,446.0,3151.0,188897.0,305289,2312302,882.0,349.0
176,2020-06-24,9229573,44918.0,7492,17351,61000,26389.0,1145906,4114.0,101963,250767.0,73572.0,2368.0,10650,12561,27936.0,51643.0,4973.0,1982.0,7155.0,161267,191449,3302.0,14540.0,13943.0,4107.0,1824,456183.0,47896.0,25391.0,21512.0,238833,18027,1111.0,1803.0,4133.0,8590.0,665.0,191410.0,49722,1166.0,2170.0,8751,27314.0,1422.0,260810.0,31825.0,32527.0,39737.0,24505.0,42432,1589.0,106108.0,247086.0,60837,31249,446.0,3156.0,190165.0,306210,2347022,885.0,349.0
177,2020-06-25,9400810,49838.0,7521,17384,61013,27487.0,1188631,4242.0,102242,254416.0,77113.0,2515.0,10777,12615,28631.0,52334.0,5150.0,1983.0,7167.0,161348,192079,3310.0,14819.0,14571.0,4114.0,1827,473105.0,49009.0,25396.0,22044.0,239410,18110,1111.0,1804.0,4140.0,8596.0,665.0,196847.0,49804,1169.0,2170.0,8777,28030.0,1528.0,264689.0,32295.0,32821.0,40104.0,24826.0,42623,1607.0,111796.0,247486.0,62324,31293,446.0,3158.0,191657.0,306862,2381361,902.0,352.0
178,2020-06-26,9582321,52444.0,7558,17431,61106,28503.0,1228114,4408.0,102611,259064.0,80599.0,2684.0,10870,12636,29141.0,53156.0,5336.0,1984.0,7172.0,161348,192556,3321.0,15619.0,15366.0,4123.0,1830,490401.0,50187.0,25405.0,22400.0,239706,18197,1111.0,1806.0,4151.0,8600.0,668.0,202951.0,49914,1170.0,2170.0,8793,29037.0,1569.0,268602.0,33069.0,33119.0,40415.0,25286.0,42736,1630.0,118375.0,247905.0,63890,31345,447.0,3158.0,193115.0,307980,2422310,907.0,352.0
179,2020-06-27,9770954,52444.0,7595,17498,61106,29423.0,1274974,4513.0,102783,263360.0,84442.0,2836.0,11038,12675,29764.0,53856.0,5517.0,1986.0,7191.0,162936,193243,3343.0,15828.0,15994.0,4127.0,1832,508953.0,51427.0,25414.0,22800.0,239961,18297,1112.0,1808.0,4173.0,8606.0,670.0,208392.0,50005,1172.0,2170.0,8815,29905.0,1711.0,272364.0,34073.0,33395.0,40866.0,25697.0,42955,1643.0,124590.0,NaN,65137,31403,447.0,3162.0,194511.0,309360,2467837,919.0,353.0


`charts_dates` contains a list of available dates in the charts dataframe, because we need to filter the covid-19 cases dataframe only to those dates for which we have charts

In [20]:
charts_dates = [str(row.day) for row in charts_df.select("day").distinct().collect()]
charts_dates[:10]

Out[14]: ['2019-08-08',
 '2019-08-22',
 '2019-04-25',
 '2017-05-11',
 '2018-08-23',
 '2018-12-20',
 '2019-01-17',
 '2019-01-24',
 '2020-02-13',
 '2018-05-03']

This is the final total covid-19 cases by country dataframe, we will use it later when we join this data with the analysis of audio feautures

In [22]:
cases_df = cases_df[cases_df.date.isin(charts_dates)]
cases_df = cases_df.fillna(0)
cases_df.head()

,date,global,ar,au,at,be,bo,br,bg,ca,cl,co,cr,cz,dk,do,ec,sv,ee,fi,fr,de,gr,gt,hn,hu,is,in,id,ie,il,it,jp,lv,lt,lu,my,mt,mx,nl,nz,ni,no,pa,py,pe,ph,pl,pt,ro,sg,sk,za,es,se,ch,tw,th,tr,gb,us,uy,vn
2,2020-01-02,27,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0
9,2020-01-09,59,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0
16,2020-01-16,61,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,0,0,0.0,0.0
23,2020-01-23,631,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,1.0,4.0,0.0,0,1,0.0,0.0
30,2020-01-30,7823,0.0,6,0,0,0.0,0,0.0,3,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,1.0,5,4,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0,11,0.0,0.0,0.0,7.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0,0,8.0,14.0,0.0,0,5,0.0,2.0


### **Charts Data Processing**

In this section we will define functions that will be helpful to process the charts dataframe.

First, we need to extract the spotify song's ids from the json object in the charts dataframe, we do it in this function which the output is a list of the song's ids in the chart

In [24]:
#UDF to get the ids separated with space as a document
from pyspark.sql.functions import udf

@udf(T.ArrayType(T.StringType()))
def get_ids(songs):
  return [s["id"] for s in songs]

charts_df = charts_df.withColumn("songs_array", get_ids(charts_df.songs))

In [25]:
charts_df.count()

Out[17]: 10981

In [26]:
weeks = charts_df.select("day").distinct().orderBy("day", ascending=False)
weeks.show(truncate=False)

+----------+
day |
+----------+
2020-06-25|
2020-06-18|
2020-06-11|
2020-06-04|
2020-05-28|
2020-05-21|
2020-05-14|
2020-05-07|
2020-04-30|
2020-04-23|
2020-04-16|
2020-04-09|
2020-04-02|
2020-03-26|
2020-03-19|
2020-03-12|
2020-03-05|
2020-02-27|
2020-02-20|
2020-02-13|
+----------+
only showing top 20 rows

In this project we need to identify the most relevant songs that caracterize each country. The approach followed to measure the importance of songs is **TF-IDF**. In these case, each chart is an array of song ids, so it will be interpreted as a document and all the charts will be the corpus.

The advantage of applying the TF-IDF approach is that it will assign lower scores to the common songs across the corpus. Which means that if there is a very popular song that appear in most of the charts the score is low because it is not useful to characterize a specific country. On the other hand, it will assign higher scores to certain songs that appear in particular countries because those songs define the specific preferences of the country.

This function uses the MLib library to compute the TF-IDF score and returns not only a dataframe with the scores, but also the vocabulary, that corresponds to the unique songs across the all the charts

In [28]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

def get_tfidf(df):
  cv = CountVectorizer(inputCol="songs_array", outputCol="features").fit(df)
  cv_result = cv.transform(df)
  idf = IDF(inputCol="features", outputCol="tfidf").fit(cv_result)
  tfidf = idf.transform(cv_result)
  return tfidf, cv.vocabulary

The output of the TF-IDF is a sparse vector. This UDF is used to build a map in which the key is the song id and the value is the TF-IDF score

In [30]:
@udf(T.MapType(T.StringType(), T.DoubleType()))
def map_weights(tfidf_vector, vocab):
  vocab = vocab.split(",")
  values = tfidf_vector.values
  indices = tfidf_vector.indices
  map_idf = {vocab[i]: float(v) for i, v in zip(indices, values)}
  return map_idf

The TF-IDF score will be used to computed a weighted average across the audio features. So each song will have an importance in the total average of audio features

In [32]:
@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def weighted_average(audio_feaures, weight):
  #Compute weighted average
  return sum(audio_feaures * weight) / sum(weight)

Applying the pleasure arousal model to identify emotions, this function plots the energy and valence of the data summarized by country

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from adjustText import adjust_text

def pleasure_arousal_plot(summary_df, date, plot_in_notebook=False):
  if plot_in_notebook:
    plt.ion()
  else:
    plt.ioff()
  x = summary_df["valence"].values
  y = summary_df["energy"].values
  fig, ax = plt.subplots(figsize=(13, 6), dpi=100)
  fig.suptitle(date)
  ax1 = plt.subplot2grid((1,2), (0,0)) 
  ax1.scatter(x, y, c=summary_df["cases"].values, cmap='Reds', edgecolors='black', linewidths=.5, s=50)

  ax1.axhline(y=0.5, xmin=0.0, xmax=1.0, color='r', linewidth=0.3)
  ax1.axvline(x=0.5, ymin=0.0, ymax=1.0, color='r', linewidth=0.3)
  ax1.set_ylabel('energy')
  ax1.set_xlabel('valence')
  ax1.set_ylim(0.2, 0.8)
  ax1.set_xlim(0.2, 0.8)

  ax2 = plt.subplot2grid((1,2), (0,1)) 
  ax2.scatter(x, y, c=summary_df["cases"].values, cmap='Reds', edgecolors='black', linewidths=.5, s=100)
  ax2.set_ylabel('energy')
  ax2.set_xlabel('valence')
  ax2.axhline(y=0.5, xmin=0.0, xmax=1.0, color='r', linewidth=0.3)
  ax2.axvline(x=0.5, ymin=0.0, ymax=1.0, color='r', linewidth=0.3)

  texts = []
  for a, b, s in zip(x, y, summary_df.index.values):
    texts.append(ax2.text(a, b, s))
  
  adjust_text(texts, lim=50)
  plt.savefig('/dbfs/FileStore/plot_'+ date +".png")

In this function we combine the audio features data summarized by country and combine the covid-19 cases for a specific date. In addition it normalize the cases by mean, it will be useful to assign a dark color to the most affected countries by week

In [36]:
def combine_covid_data(summary_df, date):
  week_covid_df = cases_df[cases_df.date == date].transpose()
  if len(week_covid_df.columns) > 0:
    week_covid_df.rename(columns={week_covid_df.columns[0]: "cases" }, inplace = True)
  else:
    week_covid_df['cases'] = 0
  join_covid_df = summary_df.set_index("country").join(week_covid_df)
  join_covid_df.drop("hk" , inplace=True) # because we don't have data for cases in hk
  join_covid_df = join_covid_df[join_covid_df.index != "global"]
  join_covid_df["cases"] = (join_covid_df["cases"] - join_covid_df["cases"].mean()) / (join_covid_df["cases"].std())
  join_covid_df.loc[join_covid_df["cases"].between(join_covid_df["cases"].quantile(.90), join_covid_df["cases"].quantile(1)), "cases"] = 1
  join_covid_df = join_covid_df.fillna(0)
  return join_covid_df

The function `analyse` will go through the following steps:

1.   Filter the charts by the desired date
2.   Compute TF-IDF
3.   Join TF-IDF results and audio features 
4.   Compute weigthed average of audio features
5.   Combine the covid cases dataframe
6.   Store the results in a spark table
7.   Pleasure-arousal plot

In [38]:
def analyse(date, plot=False):
  print(date)
  charts_time_df = charts_df.filter("day = '" + date + "'")
  tfidf_df, vocab = get_tfidf(charts_time_df)
  tfidf_df = tfidf_df.withColumn("weights", map_weights(tfidf_df.tfidf, F.lit(",".join(vocab))))
  eDF = tfidf_df.select(F.explode(tfidf_df.weights).alias("id", "weight"), "country", "day")
  eDF = eDF.join(audio_df, eDF.id == audio_df.s_id,how='left').filter("s_id is not NULL")
  weighted_df = eDF.groupBy("country").agg(weighted_average(eDF["acousticness"], eDF["weight"]).alias("acousticness"),
                           weighted_average(eDF["danceability"], eDF["weight"]).alias("danceability"),
                           weighted_average(eDF["energy"], eDF["weight"]).alias("energy"),
                           weighted_average(eDF["liveness"], eDF["weight"]).alias("liveness"),
                           weighted_average(eDF["loudness"], eDF["weight"]).alias("loudness"),
                           weighted_average(eDF["speechiness"], eDF["weight"]).alias("speechiness"),
                           weighted_average(eDF["tempo"], eDF["weight"]).alias("tempo"),
                           weighted_average(eDF["valence"], eDF["weight"]).alias("valence"))
  audio_and_covid = combine_covid_data(weighted_df.toPandas(), date)
  table_name = "data_" + date.replace("-", "")
  spark.sql("DROP TABLE IF EXISTs " + table_name)
  spark.createDataFrame(audio_and_covid).coalesce(1).write.saveAsTable(table_name)
  pleasure_arousal_plot(audio_and_covid, date, plot_in_notebook=plot)

The idea in this part of the notebook is to iterate through all over the available dates and perform the batch analysis

In [40]:
for week in weeks.collect():
  analyse(week.day)

An example of the plot for a given date is shown below

In [42]:
analyse("2020-06-25", plot=True)